## Introducción a la Visión Computacional
## Tarea 4
-----------------------
<div align="right">
Fecha de Entrega: Martes 17, Mayo 2022.
</div>

1. Proponer mejoras al sistema diseñado e implementado originalmete.

2. Volver a investigar y averiguar si hay algún algoritmo entrenado o bien si se puede extender el algoritmo utilizado trabajado en la Tarea 2.

3. Escribir un informe o entregar el notebook con comentarios y documentación, incluyendo el análisis (comentando) de la comparación con los resultados de la Tarea 2. Notar que pudiera ser que el resultado es inferior al de la Tarea 2, lo importante es el comentario o discusión del por qué se mejoró o empeoraron los nuevos resultados.

# Base de datos de patentes de vehiculos de Chile

Base de datos recopilada por mi, en imagenes de Google, las cuales fueran fotografias de patentes de vehiculos chilenos, con esto me gustaria realizar un posterior analisis, obteniendo las letras y numeros para consultar en [autoseguro.gob.cl](https://www.autoseguro.gob.cl) de carabineros y policia de investigaciones, con el objetivo de verificar si un vehiculo tiene encargo por robo.

En este caso para el pre-procesamiento de imagenes, se aplica el filtro sobel, para poder obtener la deteccion de bordes, con esto podemos tener la capacidad de detectar medianamente bien las letras y numeros de las patentes. Obteniendo estos datos, podria verificar si estos estan con encargo por robo y asi poder informar que el vehiculo fue visto por ultima vez en una locacion especifica

## Definicion funciones para pipeline de procesamiento, conversion y reconocimiento de imagenes
Define funciones intermedias para ser referenciadas desde funcion de ejecucion de pipeline
- obtencion de imagenes desde classpath de proyecto ```get_image_list_from_path(path, extension)```
- pre-procesamiento de imagenes obtenidas en punto anterior con ```save_resized_images_resx_resy```
- procesamiento de imagenes con filtro sobel ```sobel_filter```
- reconocimiento de figuras en imagenes ```detect_vehicle_license_plate```
- reconocimiento optico de caracteres, para llevar las patentes reconocidas a archivo json ```ocr_vehicle_license_plate```

# Mejoras V4

Reconocimiento optico de caracteres con Tesseract

In [63]:
import os

import numpy as np
from PIL import Image
from numpy import *
from scipy.ndimage import sobel
from matplotlib import pyplot as plt
import cv2
import time
from pathlib import Path
import shutil
import pytesseract
import glob
import json

# Functiones utilitarias

In [38]:
def stopwatch(sec, function_name):
  mins = sec // 60
  sec = sec % 60
  hours = mins // 60
  mins = mins % 60
  print("Time Lapsed = {0}:{1}:{2} in function '{3}'".format(int(hours), int(mins), sec, function_name))

In [39]:
def delete_dir(path_directory):
    dir_path = Path(path_directory)
    if dir_path.exists() and dir_path.is_dir():
        shutil.rmtree(dir_path)

In [40]:
def delete_file(path_file):
    if os.path.exists(path_file):
        os.remove(path_file)

# Funciones de pipeline

In [41]:
'''
Obtiene las imagenes definidas en el classpath del proyecto, con los formatos que se establecen en el segundo parametro
    - path: direccion donde estaran alojadas las imagenes a procesar
    - extension: lista de extensiones que seran tomadas en cuenta para obtencion de imagenes
    - limit: limite de imagenes que obtendra desde el parametro path
'''
def get_image_list_from_path(path, extension, limit = -1):
    l = []
    for ex in extension:
        for f in os.listdir(path):
            if f.endswith(ex):
                l.append(os.path.join(path,f))

    if limit != -1:
        return l[:limit]

    return l

In [42]:
#img_from_path = get_image_list_from_path("patentes", [".png", ".jpeg"])

In [43]:
'''
Guarda las imagenes que ya fueron guardadas en el classpath, con una resolucion diferente
    - x: resolucion al eje x
    - y: resolucion al eje y
    - img_list: lista de imagenes que seran procesadas a la resolucion pasada en los parametros x e y
'''
def save_resized_images_resx_resy(x, y, img_list):
    os.makedirs("out/out{}x{}/patentes".format(x, y), exist_ok=True)

    for img in img_list:
        Image.open(img).convert('RGB').resize((x, y))\
            .save("out/out{}x{}/{}".format(x, y, img))

    return get_image_list_from_path("out/out{}x{}/patentes".format(x, y), [".png", ".jpeg"])

In [44]:
#resized_images_512_512 = save_resized_images_resx_resy(512, 512, img_from_path)

In [45]:
'''
De las imagenes que sean pasadas por el parametro 'img_list' (lista de paths), seran convertidas y guardadas con el filtro sobel (reconocimiento de bordes)
    - img_list: lista de imagenes que seran convertidas y guardadas con el filtro sobel
'''
def sobel_filter(img_list):

    os.makedirs("out/out_sobel/patentes", exist_ok=True)

    for img in img_list:
        im = array(Image.open(img).convert('L'), 'f')
        imx = zeros(im.shape)
        sobel(im, 1, imx)

        imy = zeros(im.shape)
        sobel(im, 0, imy)

        magnitude = sqrt(imx**2+imy**2)
        plt.imsave("out/out_sobel/{}".format("/".join(img.split("/")[-2:])), arr = magnitude, cmap = plt.get_cmap('gray'))

    return get_image_list_from_path("out/out_sobel/patentes", [".png", ".jpeg"])

In [46]:
#sobel_filter_images = sobel_filter(resized_images_512_512)

In [47]:
'''
Funcion orientada a reconocimiento de figuras en imagenes pasadas por el parametro 'img_list' (lista de paths)
Las imagenes quedaran con bordes en rojo
    - img_list: lista de imagenes que seran convertidas y guardadas con reconocimiento de figuras
'''
def detect_vehicle_license_plate(img_list):

    os.makedirs("out/out_detection/patentes", exist_ok=True)
    for img_path in img_list:

        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, threshold = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(
            threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        i = 0
        for contour in contours:
            if i == 0:
                i = 1
                continue
            approx = cv2.approxPolyDP(
                contour, 0.01 * cv2.arcLength(contour, True), True)
            cv2.drawContours(img, [contour], 0, (0, 0, 255), 5)
            M = cv2.moments(contour)
            if M['m00'] != 0.0:
                x = int(M['m10']/M['m00'])
                y = int(M['m01']/M['m00'])

        cv2.imwrite('out/out_detection/{}'.format("/".join(img_path.split("/")[-2:])), img)

In [48]:
#detect_vehicle_license_plate(sobel_filter_images)

In [74]:
'''
Funcion orientada al reconocimiento optico de caracteres, dado una lista de imagenes a utilizar
    - img_list: lista de imagenes que seran reconocidas por tesseract y listadas en el archivo out/vehicle_license_plates.json
'''
def ocr_vehicle_license_plate(img_list):
    list_license_plates = []
    predicted_license_plates = []

    d = {}

    for img_l in img_list:
        license_plate_file = img_l
        license_plate, _ = os.path.splitext(license_plate_file)

        list_license_plates.append(license_plate)

        image = cv2.imread(img_l)
        predicted_result = pytesseract.image_to_string(image, lang = 'eng', config = '--oem 3 --psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')

        filter_predicted_result = "".join(predicted_result.split()).replace(":", "").replace("-", "")
        predicted_license_plates.append(filter_predicted_result)
        d[img_l] = filter_predicted_result

    f = open("out/vehicle_license_plates.json","w")
    f.write(json.dumps(d))
    f.close()

In [75]:
#ocr_vehicle_license_plate(sobel_filter_images)

In [76]:
'''
Funcion de ejecucion de proceso de recoleccion, pre-procesamiento, procesamiento, deteccion de figuras y reconocimiento optico de caracteres (OCR) para patentes de vehiculos

    - limit: define cual es el numero de imagenes con que trabajara en todo el flujo. Por defecto es -1
    - res_x: resolucion al eje x, por defecto 512
    - res_y: resolucion al eje y, por defecto 512

NOTA: Esta funcion generara un reporte de demora en cada uno de las funciones de ejecucion,
como tambien de la demora general en el proceso

'''
def pipeline(limit = -1, res_x = 512, res_y = 512):
    start = time.time()

    elapsed_f_preconditions = None
    elapsed_f_get_image_list_from_path = None
    elapsed_f_save_resized_images_resx_resy = None
    elapsed_f_sobel_filter = None
    elapsed_f_detect_vehicle_license_plate = None
    elapsed_f_ocr_vehicle_license_plate = None

    print("-----------------------")
    print("Pre-conditions")
    try:
        start_preconditions = time.time()
        delete_dir('out')
        #delete_file('out/vehicle_license_plates.txt')
        end_preconditions = time.time()
        elapsed_f_preconditions = end_preconditions - start_preconditions
        stopwatch(elapsed_f_preconditions, 'pre-conditions block')
    except:
      print("An exception was throw executing pre-conditions block")
    print("-----------------------")

    print("Obtaining images")
    img_from_path = None
    try:
        start_f_get_image_list_from_path = time.time()
        img_from_path = get_image_list_from_path("patentes", [".png", ".jpeg"], limit)
        end_f_get_image_list_from_path = time.time()
        elapsed_f_get_image_list_from_path = end_f_get_image_list_from_path - start_f_get_image_list_from_path
        stopwatch(elapsed_f_get_image_list_from_path, 'get_image_list_from_path')
    except:
      print("An exception was throw executing 'get_image_list_from_path' function")
    print("             Objetive images: ", len(img_from_path))
    print("-----------------------")

    print("Resolution adjust to {0}x{1}".format(res_x, res_y))
    resized_images = None
    try:
        start_f_save_resized_images_resx_resy = time.time()
        resized_images = save_resized_images_resx_resy(res_x, res_y, img_from_path)
        end_f_save_resized_images_resx_resy = time.time()
        elapsed_f_save_resized_images_resx_resy = end_f_save_resized_images_resx_resy - start_f_save_resized_images_resx_resy
        stopwatch(elapsed_f_save_resized_images_resx_resy, 'save_resized_images_resx_resy')
    except:
      print("An exception was throw executing 'save_resized_images_resx_resy' function")
    print("-----------------------")

    print("Sobel filter")
    sobel_filter_images = None
    try:
        start_f_sobel_filter = time.time()
        sobel_filter_images = sobel_filter(resized_images)
        end_f_sobel_filter = time.time()
        elapsed_f_sobel_filter = end_f_sobel_filter - start_f_sobel_filter
        stopwatch(elapsed_f_sobel_filter, 'sobel_filter')
    except:
      print("An exception was throw executing 'sobel_filter' function")
    print("-----------------------")

    print("Shape Detection")
    try:
        start_f_detect_vehicle_license_plate = time.time()
        detect_vehicle_license_plate(sobel_filter_images)
        end_f_detect_vehicle_license_plate = time.time()
        elapsed_f_detect_vehicle_license_plate = end_f_detect_vehicle_license_plate - start_f_detect_vehicle_license_plate
        stopwatch(elapsed_f_detect_vehicle_license_plate, 'detect_vehicle_license_plate')
    except:
      print("An exception was throw executing 'detect_vehicle_license_plate' function")
    print("-----------------------")

    print("OCR Vehicle license plate")
    try:
        start_f_ocr_vehicle_license_plate = time.time()
        ocr_vehicle_license_plate(img_from_path)
        end_f_ocr_vehicle_license_plate = time.time()
        elapsed_f_ocr_vehicle_license_plate = end_f_ocr_vehicle_license_plate - start_f_ocr_vehicle_license_plate
        stopwatch(elapsed_f_ocr_vehicle_license_plate, 'ocr_vehicle_license_plate')
    except:
      print("An exception was throw executing 'ocr_vehicle_license_plate' function")
    print("-----------------------")
    print("")

    elapsed = [
        elapsed_f_preconditions,
        elapsed_f_get_image_list_from_path,
        elapsed_f_save_resized_images_resx_resy,
        elapsed_f_sobel_filter,
        elapsed_f_detect_vehicle_license_plate,
        elapsed_f_ocr_vehicle_license_plate
    ]

    print("Metrics execution")
    print("Max time execution: {0}".format(np.amax(elapsed)))
    print("Min time execution: {0}".format(np.amin(elapsed)))
    print("Avg time execution: {0}".format(np.average(elapsed)))
    print("Std-Dev(+-) execution: {0}".format(np.std(elapsed)))
    print("")


    end = time.time()
    stopwatch(end - start, 'pipeline')

In [77]:
pipeline()

-----------------------
Pre-conditions
Time Lapsed = 0:0:0.043212890625 in function 'pre-conditions block'
-----------------------
Obtaining images
Time Lapsed = 0:0:0.0031571388244628906 in function 'get_image_list_from_path'
             Objetive images:  94
-----------------------
Resolution adjust to 512x512
Time Lapsed = 0:0:2.3977301120758057 in function 'save_resized_images_resx_resy'
-----------------------
Sobel filter
Time Lapsed = 0:0:3.5476009845733643 in function 'sobel_filter'
-----------------------
Shape Detection
Time Lapsed = 0:0:1.3501338958740234 in function 'detect_vehicle_license_plate'
-----------------------
OCR Vehicle license plate
Time Lapsed = 0:0:26.747474908828735 in function 'ocr_vehicle_license_plate'
-----------------------

Metrics execution
Max time execution: 26.747474908828735
Min time execution: 0.0031571388244628906
Avg time execution: 5.681551655133565
Std-Dev(+-) execution: 9.503565646929996

Time Lapsed = 0:0:34.09328269958496 in function 'pipe

In [78]:
#pipeline(res_x=512, res_y=192)

# Conclusiones procesamiento con tesseract

#### Imagenes que se pudieron procesar correctamente (correspondientes a 9), puedo llegar a concluir:
las imagenes, son directas a las placas patente, debido a esto, tesseract no tiene inconveniente en reconocer los caracteres de estas, siento totalmente correctas
![](patentes/88.jpeg)
![](patentes/6.jpeg)
![](patentes/83.jpeg)
![](patentes/95.jpeg)
![](patentes/8.jpeg)
![](patentes/4.jpeg)
![](patentes/78.jpeg)
![](patentes/9.jpeg)
![](patentes/2.jpeg)

#### Imagenes confusas (correspondientes a 23), puedo llegar a concluir:
- "JDTRC78" (patentes/5.jpeg): se encontro una letra "J" en el principio de la placa patente
    ![](patentes/5.jpeg)

- "B0TH43" (patentes/11.jpeg): se encuentra un 0 en lugar de la letra D
    ![](patentes/11.jpeg)

- "0475" (patentes/53.jpeg): no se encuentran algunos caracteres y un numero
    ![](patentes/53.jpeg)

#### Conclusiones generales

- Se opta por el procesamiento de imagenes reales con tesseract, debido a que las imagenes procesadas con los filtros sobel y la deteccion de figuras, no aporta en el procesamiento con tesseract
- las imagenes que se pudieron obtener como dataset (incuso de las nuevas 34 imagenes), no son las adecuadas para que tesseract las pueda reconocer, o bien no es el mejor angulo (ya que en algunas de las patentes, se muestra daño o estan dobladas)

# Resumen resultados ejecucion flujo reconocimiento patentes de vehiculos de Chile con Tesseract

En el caso de uso actual usando Tesseract OCR para python, se pueden obtener algunos resultados satisfactorios, plasmados en el archivo de texto `vehicle_license_plates.txt`

### Resultados satisfactorios (9)
- GSBB20 (imagen creada ) patentes/88.jpeg
- BKXZ89 (imagen directa) patentes/6.jpeg
- CYHY23 (imagen directa) patentes/83.jpeg
- TK456  (imagen directa) patentes/95.jpeg
- BHGH74 (imagen directa) patentes/8.jpeg
- GWKG64 (imagen directa) patentes/4.jpeg
- XW7818 (imagen directa) patentes/78.jpeg
- VG8961 (imagen directa) patentes/9.jpeg
- GKSB78 (imagen directa) patentes/2.jpeg

## Resultados confusos o a ser procesados de otra manera (23)
- "B0TH43" patentes/11.jpeg
- "24" patentes/75.jpeg
- "RIT" patentes/34.jpeg
- "S" patentes/59.jpeg
- "F" patentes/38.jpeg
- "H7" patentes/79.jpeg
- "7" patentes/14.jpeg
- "JPC" patentes/42.jpeg
- "JDTRC78" patentes/5.jpeg
- "B" patentes/19.jpeg
- "3A5" patentes/58.jpeg
- "A" patentes/23.jpeg
- "8" patentes/74.jpeg
- "0475" patentes/53.jpeg
- "PE" patentes/65.jpeg
- "3" patentes/49.jpeg
- "3" patentes/72.jpeg
- "CT" patentes/91.jpeg
- "22" patentes/68.jpeg
- "OB7" patentes/3.jpeg
- "FID" patentes/30.jpeg
- "B" patentes/31.jpeg
- "5" patentes/77.jpeg

# Imagenes no detectadas (61)
- patentes/10.png
- patentes/56.png
- patentes/47.jpeg
- patentes/51.jpeg
- patentes/84.jpeg
- patentes/26.jpeg
- patentes/71.jpeg
- patentes/67.jpeg
- patentes/89.jpeg
- patentes/66.jpeg
- patentes/70.jpeg
- patentes/27.jpeg
- patentes/85.jpeg
- patentes/1.jpeg
- patentes/93.jpeg
- patentes/50.jpeg
- patentes/46.jpeg
- patentes/20.jpeg
- patentes/61.jpeg
- patentes/36.jpeg
- patentes/41.jpeg
- patentes/16.jpeg
- patentes/57.jpeg
- patentes/94.jpeg
- patentes/82.jpeg
- patentes/7.jpeg
- patentes/17.jpeg
- patentes/40.jpeg
- patentes/37.jpeg
- patentes/60.jpeg
- patentes/21.jpeg
- patentes/63.jpeg
- patentes/22.jpeg
- patentes/18.jpeg
- patentes/80.jpeg
- patentes/96.jpeg
- patentes/55.jpeg
- patentes/43.jpeg
- patentes/54.jpeg
- patentes/81.jpeg
- patentes/39.jpeg
- patentes/62.jpeg
- patentes/35.jpeg
- patentes/69.jpeg
- patentes/86.jpeg
- patentes/90.jpeg
- patentes/28.jpeg
- patentes/12.jpeg
- patentes/45.jpeg
- patentes/32.jpeg
- patentes/73.jpeg
- patentes/24.jpeg
- patentes/48.jpeg
- patentes/25.jpeg
- patentes/64.jpeg
- patentes/33.jpeg
- patentes/44.jpeg
- patentes/13.jpeg
- patentes/52.jpeg
- patentes/29.jpeg
- patentes/87.jpeg